In [1]:
import numpy as np
import symforce
import os

# Symforce setup
symforce.set_symbolic_api("symengine")
symforce.set_log_level("warning")
symforce.set_epsilon_to_symbol()

from symforce import codegen
from symforce.codegen import codegen_util
from symforce import ops
import symforce.symbolic as sf
from symforce.values import Values
from symforce.notebook_util import display, display_code, display_code_file

out_put_save_directory = os.getcwd()

In [2]:
t_i = sf.V3.symbolic("t_i")
Rot_i = sf.Rot3.symbolic("Rot_i")
X_i = sf.Pose3(R=Rot_i, t=t_i)

t_f = sf.V3.symbolic("t_f")
Rot_f = sf.Rot3.symbolic("Rot_f")
X_f = sf.Pose3(R=Rot_f, t=t_f)

Rot_odo = sf.Rot3.symbolic("Rot_odo")
t_odo = sf.V3.symbolic("t_odo")
s = sf.Symbol('s')
T_odo = sf.Pose3(R=s*Rot_odo, t=s*t_odo)
display(T_odo)

a_1 = sf.V3.symbolic("a_1")
a_2 = sf.V3.symbolic("a_2")
a_3 = sf.V3.symbolic("a_3")
a_4 = sf.V3.symbolic("a_4")

r_delta, r_TSTA, r_Break = sf.symbols("r_delta r_TSTA r_Break")
theta_delta, theta_TSTA, theta_Break = sf.symbols("theta_delta theta_TSTA theta_Break")
m_0, n_0, k_0, h_0 = sf.symbols("m_0 n_0 k_0 h_0")
l_0 = sf.V3.symbolic("")
l = sf.V3.symbolic("")
delta_l = sf.V3.symbolic("")

TypeError: unsupported operand type(s) for *: 'Symbol' and 'Rot3'

Odometry Error Model

In [ ]:
odo_error = X_f.inverse() * X_i * T_odo.inverse()
odo_error_in_tangent_list = odo_error.to_tangent()

print(len(odo_error_in_tangent_list))
odo_error_in_tangent = sf.V6()
odo_error_in_tangent[0] = odo_error_in_tangent_list[0]
odo_error_in_tangent[1] = odo_error_in_tangent_list[1]
odo_error_in_tangent[2] = odo_error_in_tangent_list[2]
odo_error_in_tangent[3] = odo_error_in_tangent_list[3]
odo_error_in_tangent[4] = odo_error_in_tangent_list[4]
odo_error_in_tangent[5] = odo_error_in_tangent_list[5]

6


In [ ]:
# forward kinematic
# l_0[0] = m_0 + n_0
# l_0[1] = m_0 + 2*k_0
# l_0[2] = n_0 + 2*h_0
# l[0] = r_delta*theta_delta + l_0[0]
# l[1] = r_TSTA*theta_TSTA + l_0[1]
# l[2] = r_Break*theta_Break + l_0[2]
# m = (2*l[0] + 0.5*(l[1] + l[2] - l[0]))/(l[0]**2 - 0.25*(l[2]**2 + l[0]**2 - 2*l[0]*l[2] - l[1]**2))
# n = l[0] - m
# k = 0.5*(l[1] - m)
# h = 0.5*(l[2] - n)
# t_fk = sf.V3.symbolic("t_fk")
# t_fk[0] = (m**2 - n**2 + a_2[0]**2)/(2*a_2[0])
# t_fk[1] = a_2[1] - sf.sqrt(m**2 - t_fk[0]**2)
# t_fk[2] = 0
# error_fk = t_fk - t_f

# inverse kinematic
l_0[0] = m_0 + n_0
l_0[1] = m_0 + 2*k_0
l_0[2] = n_0 + 2*h_0
m = sf.sqrt((t_f[0] - a_1[0])**2 + (t_f[1] - a_1[1])**2)
n = sf.sqrt((t_f[0] - a_2[0])**2 + (t_f[1] - a_2[1])**2)
k = sf.sqrt((t_f[0] - a_3[0])**2 + (t_f[1] - a_3[1])**2)
h = sf.sqrt((t_f[0] - a_4[0])**2 + (t_f[1] - a_4[1])**2)
l[0] = m + n
l[1] = m + 2*k
l[2] = n + 2*h
delta_l[0] = r_delta*theta_delta
delta_l[1] = r_TSTA*theta_TSTA
delta_l[2] = r_Break*theta_Break
error_ik = l - l_0 - delta_l

# Odometry Error Function

In [ ]:
def odo_error_func(Rot_i: sf.Rot3, t_i: sf.V3,
                   Rot_f: sf.Rot3, t_f: sf.V3,
                   Rot_odo: sf.Rot3, t_odo: sf.V3,
                   s: sf.Symbol,
                   epsilon: sf.Scalar):
    return odo_error_in_tangent

residual_func_codegen = codegen.Codegen.function(func=odo_error_func, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Odometry Error Function with respect to $s$

In [ ]:
def odo_error_func_wrt_s(Rot_i: sf.Rot3, t_i: sf.V3,
                         Rot_f: sf.Rot3, t_f: sf.V3,
                         Rot_odo: sf.Rot3, t_odo: sf.V3,
                         s: sf.Symbol,
                         epsilon: sf.Scalar):
                         
    return odo_error_in_tangent.diff(s)

residual_func_codegen = codegen.Codegen.function(func=odo_error_func_wrt_s, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Odometry Error Function with respect to $X_i$

In [ ]:
def odo_error_func_wrt_pos_i(Rot_i: sf.Rot3, t_i: sf.V3,
                         Rot_f: sf.Rot3, t_f: sf.V3,
                         Rot_odo: sf.Rot3, t_odo: sf.V3,
                         s: sf.Symbol,
                         epsilon: sf.Scalar):
    return odo_error_in_tangent.jacobian(X_i)

residual_func_codegen = codegen.Codegen.function(func=odo_error_func_wrt_pos_i, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Odometry Error Function with respect to $X_f$

In [ ]:
def odo_error_func_wrt_pos_f(Rot_i: sf.Rot3, t_i: sf.V3,
                         Rot_f: sf.Rot3, t_f: sf.V3,
                         Rot_odo: sf.Rot3, t_odo: sf.V3,
                         s: sf.Symbol,
                         epsilon: sf.Scalar):
    return odo_error_in_tangent.jacobian(X_f)

residual_func_codegen = codegen.Codegen.function(func=odo_error_func_wrt_pos_f, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

# Kinematic Error Function

In [ ]:
def kin_error_func(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    return error_ik
residual_func_codegen = codegen.Codegen.function(func=kin_error_func, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $a_1$

In [ ]:
def kin_error_func_wrt_a_1(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.jacobian(a_1)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_a_1, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $a_2$

In [ ]:
def kin_error_func_wrt_a_2(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.jacobian(a_2)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_a_2, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $a_3$

In [ ]:
def kin_error_func_wrt_a_3(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.jacobian(a_3)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_a_3, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $a_4$

In [ ]:
def kin_error_func_wrt_a_4(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.jacobian(a_4)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_a_4, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $X_f$

In [ ]:
def kin_error_func_wrt_X_f(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.jacobian(t_f)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_X_f, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $m_0$

In [ ]:
def kin_error_func_wrt_m_0(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.diff(m_0)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_m_0, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $n_0$

In [ ]:
def kin_error_func_wrt_n_0(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.diff(n_0)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_n_0, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $k_0$

In [ ]:
def kin_error_func_wrt_k_0(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.diff(k_0)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_k_0, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $h_0$

In [ ]:
def kin_error_func_wrt_h_0(r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   a_1: sf.V2, a_2: sf.V2, a_3: sf.V2, a_4: sf.V2,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   t_f: sf.V3) ->sf.Matrix:
    error = error_ik.diff(h_0)
    return error
residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_h_0, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)